<a href="https://colab.research.google.com/github/Neu-h/BBG/blob/main/%5BML_04%5D_regression_LAB_%EC%9D%98%ED%98%84_0620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0.회귀 분석의 종류

target(Y)
- 분류 분석 : 범주형 변수. ex. 혈액형(A, B, O, AB), 성별(F, M), 숫자(0~9)
              정해진 것 외에 다른 값이 없음
- 회귀 분석 : 연속형 변수. ex. 판매량(0,1,2,...,100,101,...)
              주어진 값보다 더 큰 값, 작은 값, 사잇값이 있을 수 있는 것.

### Simple Linear Regression
- 단순 선형 회귀
- $y = \beta_{0} + \beta_{1} x + \epsilon$
- 하나의 X 값을 가지고 Y를 예측하는 것

### Multiple Linear Regression
- 다중 회귀 (주로)
- $y = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2} + ...+ \epsilon$
- 여러 개의 X 값을 가지고 Y를 예측하는 것

### Polynomial Regression
- 다항 회귀
- 비선형 데이터 집합을 모델링 할 때 사용
- 곡선의 다항식 선을 사용함
- 과대적합이 나타나기 쉬움
- $y = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2} + \beta_{3} x_{1}^{2} + \beta_{4} x_{2}^{2} ...+ \epsilon$
- X를 다항식으로 변경하여 모델링의 데이터로 사용함

### Ridge, Lasso Regression
- 독립변수들 간에 높은 상관 관계가 있는 경우 규제화/정규화(regularization)를 적용해 모델의 복잡도를 줄임
- Ridge : 계수값(\beta)을 0 에 가깝게 만들지만 0 이 되지 않음(모든 독립변수 사용)
- Lasso : 계수값을 0 이 되게 함으로써 그에 해당하는 특성을 제외

## 1.데이터 만들기 

### 1-1.함수 만들기

In [ ]:
# X : 키, 몸무게 
# Y : BMI 예측(수치-회귀)  vs. 비만 여부 :분류

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# https://rfriend.tistory.com/284
# https://rfriend.tistory.com/352
# https://namu.wiki/w/%ED%82%A4(%EC%8B%A0%EC%B2%B4)
# https://123dok.co/document/eqodvrmz-%EC%A7%80%EC%97%AD%EC%A0%81-%EA%B1%B4%EA%B0%95%EB%B6%88%ED%8F%89%EB%93%B1%EA%B3%BC-%EA%B0%9C%EC%9D%B8-%EB%B0%8F-%EC%A7%80%EC%97%AD%EC%88%98%EC%A4%80%EC%9D%98-%EA%B1%B4%EA%B0%95%EA%B2%B0%EC%A0%95%EC%9A%94%EC%9D%B8.html

In [10]:
# bmi = 몸무게(kg) ÷ (신장(m) × 신장(m))
# 한국 표준 데이터 생성
# seedno : 랜덤 수 발생 관련
# size : sample의 개수
# startno : ID의 시작 번호
# outlier : 이상치 비율, 0.01 => 100개 정상데이터 1개 이상치
def make_sample(seedno, size, startno=0, outlier=0):
    np.random.seed(seedno)
    sizeh = size//2           # 2로 나눈 몫. 0 또는 1 (성별)
    outlier_len = int(size * outlier)
    hm = np.random.normal(loc=175, scale=5.3, size = sizeh+1 if size % 2 else sizeh).round(1)     # 정규분포(평균 175, 표준편차 5.3), 홀수이면 sizeh+1 짝수이면 sizeh
    hw = np.random.normal(loc=162, scale=5.2, size = sizeh).round(1)                              # 정규분포(평균 162, 표준편차 5.2)
    h = np.concatenate([hm, hw])
    h /= 100                       # 키 단위를 m로 변경
    
    bmi = np.random.normal(loc=23, scale=4, size=len(h))
    w = (h*h*bmi).round(1)
    bmi = (w / (h*h)).round(2)
    h *= 100                      # 키 단위를 cm로 변경

    data = np.column_stack([h, w, bmi])
    
    # 이상치 데이터 추가
    if outlier_len != 0:
      data_outlier = make_outlier(seedno, outlier_len)
      print(data_outlier, data_outlier.shape, sep='\n')
      data = np.concatenate([data, data_outlier])

    # 'ID'를 포함한 DataFrame 생성 및 반환
    data = pd.DataFrame(data, columns=['height', 'weight', 'BMI'])
    data = data.sample(frac=1)   # 데이터를 섞어줌
    data.insert(0, 'ID', np.arange(startno, startno + len(data), dtype='int'))
    return data

In [8]:
# 이상치 데이터
# bmi = #몸무게(kg) ÷ (신장(m) × 신장(m))
# 키, 몸무게가 매우 작거나 큰 데이터 생성
# seedno : 랜덤 수 발생 관련
# size : sample의 개수
def make_outlier(seedno, size):
    np.random.seed(seedno)
    sizeh = size//2
    hlow = np.random.normal(loc=120, scale=5.3, size=sizeh+1 if size % 2 else sizeh).round(1) 
    hhigh = np.random.normal(loc=220, scale=5.2, size=sizeh).round(1)
    h = np.concatenate([hlow, hhigh])
    h /= 100
    bmi = np.random.normal(loc=23, scale=10, size=len(h))
    w = (h*h*bmi).round(1)
    bmi = (w / (h*h)).round(2)
    h *= 100
    return np.column_stack([h, w, bmi])

In [12]:
# make_sample 사용연습
df = make_sample(1234,100, outlier=0.05)          # outlier 비율이 0.05,, 100개 중 5개
df.shape

[[122.5   47.8   31.85]
 [113.7   40.8   31.56]
 [127.6   27.1   16.64]
 [218.4  110.5   23.17]
 [216.3    2.7    0.58]]
(5, 3)


(105, 4)

In [9]:
df = make_outlier(1234, 5)
df

array([[122.5 ,  47.8 ,  31.85],
       [113.7 ,  40.8 ,  31.56],
       [127.6 ,  27.1 ,  16.64],
       [218.4 , 110.5 ,  23.17],
       [216.3 ,   2.7 ,   0.58]])

### 1-2.train, test 데이터

In [13]:
# [1] train, test 데이터 생성 및 파일로 저장하기
train = make_sample(1234, 100)
test = make_sample(1, 100, len(train))
Xfeatures = ['ID', 'height', 'weight']
Yfeatures = ['ID', 'BMI']
Xtrain = train[Xfeatures]
Ytrain = train[Yfeatures]
Xtest = test[Xfeatures]
Ytest = test[Yfeatures]    # 실제 시험에서는 주지 않음

Xtrain.to_csv('x_train.csv', index=False)
Xtest.to_csv('x_test.csv', index=False)
Ytrain.to_csv('y_train.csv', index=False)
Ytest.to_csv('y_test.csv', index=False)

In [14]:
# [2] 각 데이터를 파일에서 읽어오기
# X_use, X_submission, Y, Y_hidden 이름 사용
X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
Y_hidden = pd.read_csv('y_test.csv')

In [15]:
# X_use, Y를 합쳐서 dfXY 생성
dfXY = pd.merge(X_use, Y)
dfXY.shape

(100, 4)

In [16]:
# dfXY의 상관 관계 확인
dfXY.corr()

,ID,height,weight,BMI
ID,1.000000,0.140693,-0.022971,-0.108252
height,0.140693,1.000000,0.604015,0.178859
weight,-0.022971,0.604015,1.000000,0.887726
BMI,-0.108252,0.178859,0.887726,1.000000


## 2.모델링, 성능평가

### 2-1.모델링 함수

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# [1] 데이터를 분리하고, 학습 한 뒤, R2_score를 구해 출력
def Model_Train(model, XF, YF):
    # 7:3 분리, random_state=321  (주의: stratify사용 안함 -> 분류 모델에서만 사용)
    pass

### 2-2. Multiple Linear Regression
- $y = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2} + ...+ \epsilon$
- 오차(잔차)가 가장 적게 발생하는 계수, 절편 찾기

In [ ]:
# [2] outlier가 포함되지 않은 경우, LinearRegression 모델 사용  


In [ ]:
# [3] 계수들과 절편 출력해 보기


### 2-3.이상치

In [ ]:
# [4] 이상치를 포함한 데이터 생성 (outlier=0.07 사용)


In [ ]:
# [5] 이상치가 포함된 경우, LinearRegression 모델 사용


In [ ]:
# [6] 그래프 사용하여 이상치 확인


In [ ]:
# [7] 각 feature별 Q1, Q3 확인


In [ ]:
# [8] height에 대한 outlier 확인


In [ ]:
# [9] weight에 대한 outlier 확인


In [ ]:
# [10] height에 대한 outlier 제거 및 학습


In [ ]:
# [11] weight에 대한 outlier 제거 및 학습


### 2-4.Polynomial Regression
- 다항 회귀
- 비선형 데이터 집합을 모델링 할 때 사용
- 곡선의 다항식 선을 사용함
- 과대적합이 나타나기 쉬움
- $y = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2} + \beta_{3} x_{1}^{2} + \beta_{4} x_{2}^{2} ...+ \epsilon$
- X를 다항식으로 변경하여 모델링의 데이터로 사용함

In [ ]:
# sklearn.preprocessing.PolynomialFeatures(degree=2, *, 
#               interaction_only=False, include_bias=True, order='C')

In [ ]:
# [12] PolynomialFeatures 사용하여 2차항이 포함되도록 
# Xnormal을 변환 함 (PolynomialFeatures에 poly2, 변환 결과에 Xpoly2 이름 부여)


In [ ]:
# [13] Xpoly2의 shape 확인


In [ ]:
# [14] poly2의 feature_names 확인


In [ ]:
# [15] Xpoly2의 상관계수 확인 


In [ ]:
# [16] PolynomialFeatures 사용하여 3차항이 포함되도록
# Xnormal을 변환 함 (PolynomialFeatures에 poly3, 변환 결과에 Xpoly3 이름 부여)


In [ ]:
# [17] Xpoly3의 shape 확인


In [ ]:
# [18] poly3의 shape 확인


### 2-5.Ridge Regression
- sklearn.linear_model.Ridge(alpha=1.0, *, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, tol=0.001, solver='auto', positive=False, random_state=None)
- coef(회귀계수)가 작아지지만 0이 되지는 않음

In [ ]:
# alpha=0 : 규제를 주지 않음
# alpha 값이 크다는 것은 규제를 크게 주겠다는 것을 의미 => coef_ 값들이 작아짐

In [ ]:
# [19] PolynomialFeatures degree=3과 Ridge를 사용한 모델링


In [ ]:
# [20] ridge_coef 출력하여 회귀계수 확인


### 2-6.Lasso Regression
- sklearn.linear_model.Lasso(alpha=1.0, *, fit_intercept=True, normalize='deprecated', precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
- 규제강도가 커지면 회귀계수가 작아지고 0도 될 수 있다

In [ ]:
# [21] degree=3과 Lasso를 사용한 모델링


In [ ]:
# [22] lasso_coef 출력하여 회귀계수 확인


In [ ]:
# Ridge, Lasso는 overfitting 되는 데이터에 주로 사용하여 overfitting을 해결하는 용도로 사용한다

### 2-7.DecisionTree 
- sklearn.tree.DecisionTreeRegressor(*, criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0

In [ ]:
# [22] Xnormal, Ynormal 데이터에 DecisionTreeRegressor 사용


In [ ]:
# train  test   
# 1.0000 0.7371  => overfitting, 과대적합
# 과대적합 해결방법 => Ridge, Lasso, 앙상블
#                      데이터의 크기를 늘리는 것, 함수의 parameter에 값을 변경

In [ ]:
# [23] 데이터의 개수를 2000개로 늘려 DecisionTreeRegressor 사용 


### 2-8.앙상블

- sklearn.ensemble.RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
- https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [ ]:
# [24] Xnormal, Ynormal 데이터에 RandomForestRegressor 사용


In [ ]:
# [25] Xnormal을 2차식으로 만든 후 RandomForestRegressor 사용


In [ ]:
# [26] 데이터의 개수를 2000개로 늘려 RandomForestRegressor 사용


- XBGRegressor : max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective="reg:linear", booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, importance_type="gain", **kwargs) 

In [ ]:
# [27] Xnormal, Ynormal 데이터에 XGBRegressor 사용


In [ ]:
# [28] 데이터의 개수를 2000개로 늘려 XGBRegressor 사용


## 3.회귀 모델 성능평가

- $R^2$ score
  - model.score(X, y)
  - sklearn.metrics.r2_score(y_true, y_pred)
  - 분산 기반으로 예측 성능을 평가
  - 예측값 분산 / 실제값 분산
- MAE(Mean Absolute Error)
   - sklearn.metrics.mean_absolute_error(y_true, y_pred)
   - 실제 값과 예측 값의 차이를 절댓값으로 변환해 평균한 것
- MSE(Mean Squared Error)
   - sklearn.metrics.mean_squared_error(y_true, y_pred)  
   - 실제 값과 예측 값의 차이를 제곱해 평균한 것
- RMSE(Root Mean Squared Error)
   - sklearn API에 없음
   - MSE에 np.sqrt() 또는 ** 0.5를 사용함
   - MSE 값은 오류의 제곱을 구하므로 실제 오류 평균보다 더 커지는 특성이 있어 MSE에 루트를 씌운 것
- MSLE(Mean Squared Log Error)
   - sklearn.metrics.mean_squared_log_error(y_true, y_pred)
   - MSE에 로그를 적용해준 지표
- RMSLE(Root Mean Square Logarithmic Error) 
   - RMSE에 로그를 적용해준 지표
   - 이상치가 있더라도 변동폭이 크지 않음(이상치에 강건함)
   - 실제값보다 예측값이 작을 때 더 큰 패널티 부여
   - 작게 예측하면 안되는 경우 사용하면 좋음 (배달 시간 예측)
   - 실제값, 예측값에 음수가 있으면 안됨 (오류 발생)
- 참조 : https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-17-%ED%9A%8C%EA%B7%80-%ED%8F%89%EA%B0%80-%EC%A7%80%ED%91%9C

### 3-1.성능평가 함수

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

# [29] r2_score, mae, mse, rmse, msle, rmsle 를 구해 반환하는 함수를 작성합니다.
# 한 개의 행에 6가지 성능평가 결과를 소수점 아래 4자리까지 표시되도록 하여 작성합니다.
def get_scores(model, X, Y):
    pass

### 3-2.모델링 함수

In [ ]:
# [30] Model_Train함수를 수정해서 
# train, test에 대한 6가지 성능평가 결과를 출력하는 Model_Train2 함수를 작성합니다.
def Model_Train_2(model, XF, YF, scale=False):
    # 7:3 분리, random_state=321  (주의: stratify사용 안함 -> 분류 모델에서만 사용)
    if scale:
        XF = MinMaxScaler().fit_transform(XF)
    xtrain, xtest, ytrain, ytest = train_test_split(XF, YF, 
                                                    test_size=0.3, 
                                                    random_state=321)
    model.fit(xtrain, ytrain)
    #  코드 추가 부분
    
    
    return model

### 3-3.여러 모델 적용

In [ ]:
# [31] Xnormal, Ynormal 데이터 사용, LinearRegression => model1


train : r2: 0.9931 mae: 0.2648 mse: 0.1200 msle: 0.0003 rmse: 0.3464 rmsle: 0.0165
test : r2: 0.9625 mae: 0.3783 mse: 0.3626 msle: 0.0010 rmse: 0.6022 rmsle: 0.0313


LinearRegression()

In [ ]:
# [32] Xpoly3, Ynormal 데이터 사용, LinearRegression => model2


In [ ]:
# [33] Xlarge, Ylarge 데이터 사용, XGBRegressor => model3


In [ ]:
# [34] X_submission, Y_hidden 데이터를 사용하여 model1, model2, model3의 성능을 평가하여봄 
# (실제는 할 수 없음)


In [ ]:
# [35] 제출파일 만들기 (실제 문제에서 요구하는 형태로 작성해야 함)


## 4.웹사이트 방문자 예측

In [4]:
!git clone https://github.com/Soyoung-Yoon/bigdata

Cloning into 'bigdata'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 41 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


### 4-1. 데이터 이해, 생성
- 1개 데이터를 나누어 시험용으로 변경해 보는 작업
- 실제 시험에서는 이런 과정은 필요 없음


In [5]:
# https://www.kaggle.com/bobnau/daily-website-visitors
# Daily number of pages loaded
# Daily number of visitors from whose IP addresses there haven't been hits on any page in over 6 hours
# Number of unique visitors who do not have a cookie identifying them as a previous customer
# Number of unique visitors minus first time visitors

In [6]:
# [0] 사용 라이브러리 import, pd.set_option
import pandas as pd
import numpy as np
import datetime as dt

#(아래는 생략 가능)
# 데이터가 많을 때 생략되지 않도록 
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 20)
# 출력 format 지정
pd.set_option('display.float_format', '{:.4f}'.format)

In [7]:
# [1] 'bigdata/daily-website-visitors.csv' 파일 불러오기, shape 확인하기
df = pd.read_csv('bigdata/daily-website-visitors.csv')
df.shape

(2167, 8)

In [8]:
# [2] head를 사용해 데이터의 모습 확인
# 컬럼명이 대소문자 섞여 있어 사용하기 좋지 않음 
df.head(2)

,Row,Day,Day.Of.Week,Date,Page.Loads,Unique.Visits,First.Time.Visits,Returning.Visits
0,1,Sunday,1,9/14/2014,"2,146","1,582","1,430",152
1,2,Monday,2,9/15/2014,"3,621","2,528","2,297",231


In [9]:
# [3] 컬럼명 정리 (실제 시험에서는 정리되어있음) -> 소문자로 변경, '.'을 '_'로 변경
# 의현 : df.columns = ['row', 'day', 'day_of_week', 'date', 'page_loads', 'unique_visits', 'first_time_visits', 'returning_visits']
df.columns = df.columns.str.replace('.', '_', regex=True).str.lower()
display(df.head(2))

,row,day,day_of_week,date,page_loads,unique_visits,first_time_visits,returning_visits
0,1,Sunday,1,9/14/2014,"2,146","1,582","1,430",152
1,2,Monday,2,9/15/2014,"3,621","2,528","2,297",231


컬럼 정보
- row : 1번 부터 시작하는 일련번호
- day : 요일 정보
- day_of_week : 요일 정보
- date : 날짜 정보
- page_loads : 로드된 일별 페이지 수
- unique_visits : 6시간 이상 페이지에서 조회되지 않은 IP 주소의 일일 방문자 수
- first_time_visits : 이전 고객으로 식별되는 쿠키를 가지고 있지 않은 고유 방문자 수 (첫 방문자 수)
- returning_visits : unique_visits 수에서 first_time_visits 제외 (재방문자 수)

In [10]:
# [4] unique_visits를 예측하는 값으로 사용할 것이며, 수치데이터 이어야 함
# 콤마 제거 후, int로 형변환
df['page_loads'] = df['page_loads'].str.replace(',', '', regex=True).astype('int')          # str.replace() : 단일 변환만
df['unique_visits'] = df['unique_visits'].replace(',', '', regex=True).astype('int')        # replace() : 목록으로 여러 개 변환 가능
df['first_time_visits'] = df['first_time_visits'].replace(',', '', regex=True).astype('int')
df['returning_visits'] = df['returning_visits'].replace(',', '', regex=True).astype('int')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   row                2167 non-null   int64 
 1   day                2167 non-null   object
 2   day_of_week        2167 non-null   int64 
 3   date               2167 non-null   object
 4   page_loads         2167 non-null   int64 
 5   unique_visits      2167 non-null   int64 
 6   first_time_visits  2167 non-null   int64 
 7   returning_visits   2167 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 135.6+ KB
None


In [11]:
# [5] 시험 형식으로 train, test 데이터로 나누어 저장합니다.  (6:4 비율)  -> 시험에서는 데이터 주어지므로 안해도 됨
# 의현:
#from sklearn.model_selection import train_test_split
#X = df.drop(columns=['unique_visits'])
#Y = df[['row', 'unique_visits']]
#xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.4)
train_size = len(df) - int(len(df)*0.4)
# 여러 가지 데이터가 고루 섞이도록 해야 함
df = df.sample(frac=1,              # 전체를 반환 
               random_state=1234)
df['row'] = range(1, len(df) + 1)   # 섞였을테니 재배정
train = df.iloc[:train_size, :]
test = df.iloc[train_size:, :]
print(train.shape, test.shape)

y = 'unique_visits'
X_train = train.drop(columns=y)
Y_train = train[['row', y]]
X_test = test.drop(columns=y)
Y_test = test[['row', y]]
X_train.to_csv('x_train.csv', index=False)
Y_train.to_csv('y_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
Y_test.to_csv('y_test.csv', index=False)

(1301, 8) (866, 8)


### 4-2. 데이터 불러오기, 전처리

In [12]:
# [1] 데이터 파일 불러오기
X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
# 시험엔 없음
Y_hidden = pd.read_csv('y_test.csv')

In [35]:
# [2] dfX 만들기, dfX의 info() 확인 -> ignore_index=True
# use, submission 데이터를 묶어 전처리 하기 위함
dfX = pd.concat([X_use, X_submission], ignore_index=True)
print(dfX.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   row                2167 non-null   int64 
 1   day                2167 non-null   object
 2   day_of_week        2167 non-null   int64 
 3   date               2167 non-null   object
 4   page_loads         2167 non-null   int64 
 5   first_time_visits  2167 non-null   int64 
 6   returning_visits   2167 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 118.6+ KB
None


In [36]:
# [3] dfX의 object 값이 어떤 것인지 확인하기 위해 5줄 출력을 합니다.
dfX.head()

,row,day,day_of_week,date,page_loads,first_time_visits,returning_visits
0,1,Friday,6,6/26/2020,3555,2248,427
1,2,Thursday,5,8/29/2019,4221,2477,505
2,3,Friday,6,11/18/2016,4767,2734,690
3,4,Saturday,7,11/30/2019,3301,2053,390
4,5,Sunday,1,7/5/2020,2948,1836,362


In [37]:
# [4] 'page_loads', 'first_time_visits', 'returning_visits' 에 대해서 
#  콤마를 없애고, int로 형변환 합니다
cols = ['page_loads', 'first_time_visits', 'returning_visits']
dfX[cols] = dfX[cols].replace('.', '', regex=True).astype(int)       # str.replace()는 Series에서만 가능
dfX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2167 entries, 0 to 2166
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   row                2167 non-null   int64 
 1   day                2167 non-null   object
 2   day_of_week        2167 non-null   int64 
 3   date               2167 non-null   object
 4   page_loads         2167 non-null   int64 
 5   first_time_visits  2167 non-null   int64 
 6   returning_visits   2167 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 118.6+ KB


In [42]:
# [5] 'date'에 대해서 datetime64로 형변환 합니다. -> type만 확인하는게 아니라 변환된 내용이 맞는지도 확인하기
# dfX['date'] = dfX['date'].astype('datetime64')   -> 이거로 안되면 아래처럼
dfX['date'] = pd.to_datetime(dfX['date'], format='%m/%d/%Y')   
dfX.head()

,row,day,day_of_week,date,page_loads,first_time_visits,returning_visits
0,1,Friday,6,2020-06-26,3555,2248,427
1,2,Thursday,5,2019-08-29,4221,2477,505
2,3,Friday,6,2016-11-18,4767,2734,690
3,4,Saturday,7,2019-11-30,3301,2053,390
4,5,Sunday,1,2020-07-05,2948,1836,362


In [17]:
# [6] 형변환이 올바르게 되었는지 확인합니다.
dfX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2167 entries, 0 to 865
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   row                2167 non-null   int64         
 1   day                2167 non-null   object        
 2   day_of_week        2167 non-null   int64         
 3   date               2167 non-null   datetime64[ns]
 4   page_loads         2167 non-null   int64         
 5   first_time_visits  2167 non-null   int64         
 6   returning_visits   2167 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 135.4+ KB


In [18]:
# [7] day, day_of_week를 확인해 보면 동일한 데이터 인 것을 알 수 있음
# 'day' 컬럼을 제거합니다.
dfX2 = dfX.drop(columns=['day'])

In [44]:
# [8] 'date'에서 'year', 'month', 'day'에 대한 정보를 dfX3에 포함시키고, 'date' 를 제거합니다.
# 상황에 따라 'quarter' 사용도 고려할 수 있음, 주말/주중으로 나눌 수도 있음 ...  
dfX3 = dfX2.copy()
dfX3['year'] = dfX3['date'].dt.year
dfX3['month'] = dfX3['date'].dt.month
dfX3['day'] = dfX3['date'].dt.day
dfX3 = dfX3.drop(columns=['date'])
dfX3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2167 entries, 0 to 865
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   row                2167 non-null   int64
 1   day_of_week        2167 non-null   int64
 2   page_loads         2167 non-null   int64
 3   first_time_visits  2167 non-null   int64
 4   returning_visits   2167 non-null   int64
 5   year               2167 non-null   int64
 6   month              2167 non-null   int64
 7   day                2167 non-null   int64
dtypes: int64(8)
memory usage: 152.4 KB


In [46]:
# [9] dfX3과 Y를 merge 합니다.
# Y와 X의 feature의 관계를 분석해 보기 위해!
dfXY = pd.merge(dfX3, Y)   # merge: 같은 rowid를 갖는 행끼리 병합. dfXY는 결국 train data로만 이루어져 있는 것
display(dfXY.head(2))

,row,day_of_week,page_loads,first_time_visits,returning_visits,year,month,day,unique_visits
0,1,6,3555,2248,427,2020,6,26,2675
1,2,5,4221,2477,505,2019,8,29,2982


In [48]:
# [10] dfXY의 상관계수를 확인한다
# 'unique_visits'와 상관이 매우 높은 feature들이 있음 (page_loads, first_time_visits, returning_visits) - 자기들끼리도 상관관계 높음
# -> 모든 변수를 사용하지 않고 상관관계 높은 변수만 몇개 사용해도 성능 좋을 것으로 예상됨
dfXY.corr()['unique_visits']

row                  0.0162
day_of_week         -0.2592
page_loads           0.9885
first_time_visits    0.9962
returning_visits     0.9059
year                 0.0707
month               -0.0478
day                 -0.0353
unique_visits        1.0000
Name: unique_visits, dtype: float64

In [22]:
# [11] 'day_of_week' 별 'unique_visits'의 평균을 구해본다.
# 2, 3, 4, 5 가 높고 1, 6, 7이 낮은 것을 볼 수 있다.
A = dfXY.groupby('day_of_week')['unique_visits'].mean()
print(A)

day_of_week
1   2351.3175
2   3487.1016
3   3590.0156
4   3454.6684
5   3362.2404
6   2680.1477
7   1784.1872
Name: unique_visits, dtype: float64


In [23]:
type(A)

pandas.core.series.Series

In [49]:
# [12] 'day_of_week' 별 평균이 높은 것에 대한 데이터를 high_day_of_week로 저장
high_day_of_week = A[A > 3000].index.values
print(high_day_of_week)   # array

[2 3 4 5]


In [51]:
# [13] 'month' 별 'unique_visits'의 평균을 구해본다.
# 2, 3, 4, 5, 10, 11이 높고 1, 6, 7이 낮은 것을 볼 수 있다.
B = dfXY.groupby('month')['unique_visits'].mean()
print(B)

month
1    2504.1947
2    3164.8842
3    3306.5798
4    3835.3818
5    3324.8396
6    2619.0339
7    2315.3505
8    2266.5980
9    2668.1776
10   3188.5345
11   3445.1504
12   2776.1048
Name: unique_visits, dtype: float64


In [52]:
# [14] 'month' 별 평균이 높은 것에 대한 데이터를 high_month로 저장
high_month = B[B>3000].index.values
print(high_month)

[ 2  3  4  5 10 11]


In [54]:
# [15] high_day_of_week, high_month의 index 정보를 사용해 파생 변수 생성
# dfX3에 높은 평균을 갖는 요일과 달을 1, 아닌 것을 0으로 하는 dow_h, month_h 변수 추가  - train/test data에 동일한 전처리!! <dfX3>
import numpy as np
# 의현: dfX3['dow_h'] = 1 if dfX3['day_of_week'].values() in dow_h['day_of_week'] else 0
dfX3['dow_h'] = np.where(dfX3['day_of_week'].isin(high_day_of_week), 1, 0)      # np.where(조건, 참일 때 값/배열, 거짓일 때 값/배열) : numpy 조건식
dfX3['month_h'] = np.where(dfX3['month'].isin(high_month), 1, 0)                # A.isin(B) : A의 값이 B에 있으면 참, 없으면 거짓
# dfX3['dow_h'] = dfX3['day_of_week'].isin(high_day_of_week).astype(int)        # 이렇게 해도 동일. 참/거짓 -> 1/0
display(dfX3.head())

,row,day_of_week,page_loads,first_time_visits,returning_visits,year,month,day,dow_h,month_h
0,1,6,3555,2248,427,2020,6,26,0,0
1,2,5,4221,2477,505,2019,8,29,1,0
2,3,6,4767,2734,690,2016,11,18,0,1
3,4,7,3301,2053,390,2019,11,30,0,1
4,5,1,2948,1836,362,2020,7,5,0,0


In [61]:
C = dfXY.groupby('year')['unique_visits'].mean()
high_year = C[C>2900].index.values.astype(int)
print(C, '\nhigh_year : ', high_year)

year
2014   2588.9322
2015   2946.1948
2016   3240.0045
2017   2502.0631
2018   3027.3835
2019   3069.0755
2020   3155.5333
Name: unique_visits, dtype: float64 
high_year :  [2015 2016 2018 2019 2020]


In [63]:
dfX3['yr_h'] = np.where(dfX3['year'].isin(high_year), 1, 0)
dfX3.head()

,row,day_of_week,page_loads,first_time_visits,returning_visits,year,month,day,dow_h,month_h,yr_h
0,1,6,3555,2248,427,2020,6,26,0,0,1
1,2,5,4221,2477,505,2019,8,29,1,0,1
2,3,6,4767,2734,690,2016,11,18,0,1,1
3,4,7,3301,2053,390,2019,11,30,0,1,1
4,5,1,2948,1836,362,2020,7,5,0,0,1


### 4-3.성능평가, 모델링 함수


In [ ]:
#[16] 사용할 라이브러리 import
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle

# [17] r2_train, r2_test, mae, mse, rmse, msle, rmsle 를 구해 
#      Series로 반환하는 함수를 작성합니다.
def get_scores2(model, xtrain, xtest, ytrain, ytest):
    scores = None
    
    return  scores

In [ ]:
# [18] 다양한 모델을 만들고 성능을 출력하는 함수 작성
# 결과를 DataFrame으로 만듭니다
def make_models(xtrain, xtest, ytrain, ytest, n=300, RL=False):
    temp = pd.DataFrame()
    
    return temp

### 4-4.데이터 분리, 모델적용

In [ ]:
# [19] 학습에 사용할 feature 선택, 
# dfX4로 저장
# 'page_loads','first_time_visits' => 이것을 사용하면 성능이 무지 좋아요. (높은 상관계수)



In [ ]:
# [20] dfX4 데이터를 X_use, X_submission 으로 다시 분리
# dfX4 분리에서는 X_use의 행의 개수 사용
# YF, Y_submissionF 생성  Y에서 'unique_visits'만 선택


In [ ]:
# [21] XF, YF 데이터를 사용하여 데이터를 분리하고 make_models를 호출합니다.


In [ ]:
# [22] 여러 가지 조건으로 정렬하고 평가할 수 있음


In [ ]:
# [23] Polynomial Regression  + Ridge, Lasso 실행


In [ ]:
# [24] 여러 가지 조건으로 정렬하고 평가할 수 있음


In [ ]:
# [TIP] 학습 중 warning 메시지 없애기
import warnings
warnings.filterwarnings('ignore')

### 4-5. 모델 선택, 결과 제출

In [ ]:
# [25] xtrain, ytrain으로 모델 학습시켜 model 로 저장


In [ ]:
# [26] X_submissionF 데이터로 pred 구하고 submission DataFrame 생성


In [ ]:
# [27] xtrain2, ytrain2으로 모델 학습시켜 model 로 저장


In [ ]:
# [28] X_submissionF2 데이터로 pred 구하고 submission DataFrame 생성


In [ ]:
# [29] 파일로 저장

### 4-6.시험보러 가서는 이렇게
- 직접 코드를 작성해 보세요.

In [ ]:
# 여기에 작성하시면 됩니다.
# 영상에서 make_models_f 함수 내부의 
# get_scores2 를  get_scores_f 로 변경을 안했네요 ^^! 수정해 주세요!





## 5.보험 예측

In [ ]:
!git clone https://github.com/Soyoung-Yoon/bigdata

Cloning into 'bigdata'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 41 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


### 5-1. 데이터 이해, 생성
- 1개 데이터를 나누어 시험용으로 변경해 보는 작업
- 실제 시험에서는 이런 과정은 필요 없음


In [ ]:
# https://www.kaggle.com/qusaybtoush/insurance-forecast-by-using-linear-regression
# Age: age of primary beneficiary
# Sex: insurance contractor gender, female, male
# bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
# Children: Number of children covered by health insurance / Number of dependents
# Smoker: Smoking
# Region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
# Charges: Individual medical costs billed by health insurance

컬럼 정보
- age : 1차 수혜자 연령
- sex : 보험 계약자 성별
- bmi : 비만도 지수
- children : 건강보험 적용 대상 아동 수
- smoker : 흡연여부
- regioin : 거주지역
- charges : 의료 보험에서 청구하는 개별 의료비

In [ ]:
# [0] 사용 라이브러리 import
import pandas as pd

# 데이타가 많은경우 모두 출력 안되고 ... 으로 생략해서 출력됨.
pd.set_option('max_rows',500)    #출력할 max row를 지정
pd.set_option('max_columns',20)  #출력할 max columns를 지정
#출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
# [1] 'bigdata/insurance.csv' 파일 불러오기, shape 확인하기
df = pd.read_csv('bigdata/insurance.csv')
df.shape

(1338, 7)

In [ ]:
# [2] head를 사용해 데이터의 모습 확인  (Y => charges)
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.9000,0,yes,southwest,16884.9240
1,18,male,33.7700,1,no,southeast,1725.5523


In [ ]:
# [3] 시험 형식으로 train, test 데이터로 나누어 저장합니다.  (6:4 비율)

train_size = len(df) - int(len(df)*0.4)

# 여러 가지 데이터가 고루 섞이도록 해야 함
df = df.sample(frac=1, random_state=1234)
df.insert(0, 'ID', range(1, len(df)+1))

train = df.iloc[:train_size, ]
test = df.iloc[train_size:, ]
print(train.shape, test.shape)
ycolumn = 'charges'
X_train = train.drop(columns=ycolumn)
Y_train = train[['ID', ycolumn]]
X_test = test.drop(columns=ycolumn)
Y_test = test[['ID', ycolumn]]
X_train.to_csv('x_train.csv', index=False)
Y_train.to_csv('y_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
Y_test.to_csv('y_test.csv', index=False)

(803, 8) (535, 8)


### 5-2. 데이터 불러오기, 전처리

In [ ]:
# [1] 데이터 파일 불러오기
X_use = pd.read_csv('x_train.csv')
X_submission = pd.read_csv('x_test.csv')
Y = pd.read_csv('y_train.csv')
Y_hidden = pd.read_csv('y_test.csv')

In [ ]:
# [2] dfX 만들기, dfX의 info() 확인
# use, submission 데이터를 묶어 전처리 하기 위함


In [ ]:
# [3] 컬럼별 고윳값 데이터 수 개수 확인


In [ ]:
# [4] object 컬럼들에 대해서 Encoding 실행 - 필수


In [ ]:
# [5] dfX와 Y를 합하여 dfXY 생성


In [ ]:
# [6] 'charges'와 다른 컬럼의 상관계수를 확인합니다.   (dfXY의 모든 것 확인해도 좋음)


### 5-3. 성능평가, 모델링 함수
- 4-3의 것 그대로 사용

In [ ]:
#  4-3 에서 [16] [17] [18] 실행

### 5-4. 데이터 분리, 모델적용

In [ ]:
# [7] X의 최종 전처리 버전인 dfX의 컬럼 확인


In [ ]:
# [8] dfX 에서 사용할 feature 선택하여 dfX2로 저장


In [ ]:
# [9] dfX2 데이터를 XF, X_submissionF 로 다시 분리
# dfX2 분리에서는 X_use의 행의 개수 사용
# Y에서 'charges' 만 YF로 사용


In [ ]:
# [10] XF, YF 데이터를 사용하여 데이터를 분리하고 make_models를 호출합니다.


In [ ]:
# [11] models를 평가기준으로 정렬하여 좋은 모델을 알아봄


### 5-5. 모델 선택, 결과 제출

In [ ]:
# [12] 모델 학습시켜 model 로 저장, submission 만들기, 저장하기


In [ ]:
# [13] Y_submissionF 를 사용하여 성능을 알아봄 (필수 아님)


### 5-6.시험보러 가서는 이렇게

In [ ]:
# 직접 작업해 보세요!


